<a id='top'></a>
# Chicago Ridesharing

#### Contributors: Muoyo Okome, Anesu Masube

<a id='toc'></a>
### Table of Contents
1. [Problem Statement](#problemstatement)
2. [Data Sources](#datasources)
3. [Data Cleaning](#datacleaning)
4. [Linear Regression](#regression)  
5. [Findings](#findings)
6. [Next Steps](#nextsteps)

In [5]:
# Import necessary libraries
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

In [7]:
# Import utility functions
%run ../python_files/utils

<a id='problemstatement'></a>
### Problem Statement

**Can data help ridesharing drivers earn more?**

The key question we look to answer is whether knowing where a ridesharing ride was initiated and what time it was initiated can help us to predict the fare for that ride.

Eventually our goal is to provide these insights to ridesharing drivers as a service to help them choose the best driving schedules and waiting positions to optimize their earnings.

[Back to Top ↑](#top)

<a id='datasources'></a>
### Data Sources

#### **[City of Chicago Data Portal](https://data.cityofchicago.org/Transportation/Transportation-Network-Providers-Trips/m6dm-c72p)**

- All trips, starting November 2018, reported by Transportation Network Providers (sometimes called rideshare companies) to the City of Chicago as part of routine reporting required by ordinance.

- Census Tracts are suppressed in some cases, and times are rounded to the nearest 15 minutes. Fares are rounded to the nearest `$`2.50 and tips are rounded to the nearest `$`1.00. 

- 101 million rows of data!

#### **[Dark Sky Weather API](https://darksky.net/dev/docs#time-machine-request)**

The Dark Sky API allows you to look up the weather anywhere on the globe, returning (where available):

- Current weather conditions
- Minute-by-minute forecasts out to one hour
- Hour-by-hour and day-by-day forecasts out to seven days
- Hour-by-hour and day-by-day observations going back decades
- Severe weather alerts in the US, Canada, European Union member nations, and Israel


<br> 

[Back to Top ↑](#top)

<a id='datacleaning'></a>
### Data Cleaning

... Insert Narrative ...

In [8]:
# Read in original data
df = get_trip_records(limit=1000000)

... Insert Narrative ...

In [3]:
# Clean data & load into final dataframe
%run ../python_files/data_cleaning

In [4]:
df = clean_data(df)

In [9]:
# Import python files we've created to help
%run ../python_files/regression
%run ../python_files/visualizations



[Back to Top ↑](#top)

<a id='regression'></a>
### Linear Regression

We selected a significance level of $\alpha=$0.05 for all hypothesis tests.

... Insert Narrative ...

<a id='test1'></a>
### Test 1

... Insert Narrative ...

In [2]:
# Test 1

### Test 1 Results

... Insert Narrative ...

In [4]:
# Visualization

[Back to Top ↑](#top)

<a id='test2'></a>
### Test 2

... Insert Narrative ...

In [8]:
# Test 2

### Test 2 Results

... Insert Narrative ...

In [3]:
# Visualization

[Back to Top ↑](#top)

<a id='findings'></a>
### Findings

1. Drivers can earn the most at 5 am. Peak fares occur between 4-6am on a daily basis.

2. Airport pickups lead to higher earnings.

3. Strong relationship between temperature and trip fare.


[Back to Top ↑](#top)

<a id='nextsteps'></a>
### Next Steps

1. Further refine the model to provide recommendations of ideal pickup location given time & weather (deliver to drivers via app?)

2. Finding the correct independent variables to increase model’s predictive power (events/occasions)

3. How can we extend our model to help different groups such as riders & competing rideshare companies?


[Back to Top ↑](#top)